In [ ]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import month, year, col,sum
from snowflake.snowpark.version import VERSION
from snowflake.core import Root
from snowflake.core.task import Task, StoredProcedureCall
import json 
import logging 

session = get_active_session()
print(session)

In [ ]:
snow_df_spend = session.table('campaign_spend')
snow_df_spend.queries

In [ ]:
with session.query_history() as history:
    snow_df_spend.show(20)
history.queries

In [ ]:
snow_df_spend_per_channel = snow_df_spend.group_by(year('DATE'), month('DATE'), 'CHANNEL').agg(sum('TOTAL_COST').as_('TOTAL_COST')).\
with_column_renamed('"YEAR(DATE)"', "YEAR").with_column_renamed('"MONTH(DATE)"', "MONTH").sort('YEAR','MONTH')

snow_df_spend_per_channel.show(10)

In [ ]:
snow_df_spend_per_month = snow_df_spend_per_channel.pivot('CHANNEL', ['search_engine', 'social_media','video','email']).sum('TOTAL_COST').sort('YEAR','MONTH')
snow_df_spend_per_month = snow_df_spend_per_month.select(
    col("YEAR"),
    col("MONTH"),
    col("'search_engine'").as_("SEARCH_ENGINE"),
    col("'social_media'").as_("SOCIAL_MEDIA"),
    col("'video'").as_("VIDEO"),
    col("'email'").as_("EMAIL")
)
snow_df_spend_per_month.show()

In [ ]:
snow_df_spend_per_month.write.mode('overwrite').save_as_table('SPEND_PER_MONTH')